<a href="https://colab.research.google.com/github/Puthalir/Puthalir.github.io/blob/main/Day_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 12.9 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

# Initialize the WebDriver
#driver = webdriver.Chrome()  # or webdriver.Firefox() for Firefox

# Base URL components
base_url = "https://www.indeed.com/jobs"
query = "data+scientist"
location = "Bolton%2C+MA"

# List to store job data
job_listings = []

# Iterate over the first 5 pages
for page in range(0, 50, 10):  # Indeed shows 10 jobs per page
    url = f"{base_url}?q={query}&l={location}&start={page}"
    driver.get(url)
    time.sleep(3)  # Wait for the page to load

    # Find all job cards on the page
    job_cards = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')

    for card in job_cards:
        try:
            title = card.find_element(By.CLASS_NAME, 'jobTitle').text
        except NoSuchElementException:
            title = ''
        try:
            company = card.find_element(By.CLASS_NAME, 'companyName').text
        except NoSuchElementException:
            company = ''
        try:
            location = card.find_element(By.CLASS_NAME, 'companyLocation').text
        except NoSuchElementException:
            location = ''
        try:
            summary = card.find_element(By.CLASS_NAME, 'job-snippet').text
        except NoSuchElementException:
            summary = ''

        job_listings.append({
            'Title': title,
            'Company': company,
            'Location': location,
            'Summary': summary
        })

# Close the WebDriver
driver.quit()

# Save the data to a CSV file
df = pd.DataFrame(job_listings)
df.to_csv('indeed_jobs.csv', index=False)
print("Job listings have been saved to indeed_jobs.csv")


NameError: name 'driver' is not defined

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import tempfile

# Setup Chrome options with a temporary profile to avoid session error
chrome_options = Options()
chrome_options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")  # avoids profile conflict
chrome_options.add_argument("--headless")  # run in background
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set path to chromedriver if not in PATH
# Example: Service("/path/to/chromedriver")
service = Service()

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Base URL
base_url = "https://www.indeed.com/jobs?q=data+scientist&l=Bolton%2C+MA&from=searchOnHP%2Cwhatautocomplete%2Cwhereautocomplete&vjk=2c15f6f156f07bdb"
query = "data+scientist"
location = "Bolton%2C+MA"

# List to store job data
job_listings = []

# Scrape first 5 pages (10 jobs per page)
for page in range(0, 50, 10):
    url = f"{base_url}?q={query}&l={location}&start={page}"
    driver.get(url)
    time.sleep(3)

    job_cards = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')

    for card in job_cards:
        try:
            title = card.find_element(By.CLASS_NAME, 'jobTitle').text
        except NoSuchElementException:
            title = ''
        try:
            company = card.find_element(By.CLASS_NAME, 'companyName').text
        except NoSuchElementException:
            company = ''
        try:
            location = card.find_element(By.CLASS_NAME, 'companyLocation').text
        except NoSuchElementException:
            location = ''
        try:
            summary = card.find_element(By.CLASS_NAME, 'job-snippet').text
        except NoSuchElementException:
            summary = ''

        job_listings.append({
            'Title': title,
            'Company': company,
            'Location': location,
            'Summary': summary
        })

# Close the driver
driver.quit()

# Save to CSV
df = pd.DataFrame(job_listings)
df.to_csv('indeed_jobs.csv', index=False)
print("Scraping complete. Saved to 'indeed_jobs.csv'.")


Scraping complete. Saved to 'indeed_jobs.csv'.


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# Define headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Base URL components
base_url = "https://www.indeed.com/jobs"
query_params = {
    "q": "data scientist",
    "l": "Bolton, MA",
    "start": 0
}

# Open a file to write the job listings
with open("indeed_jobs.txt", "w", encoding="utf-8") as file:
    for page in range(5):  # Iterate over the first 5 pages
        query_params["start"] = page * 10  # Indeed shows 10 jobs per page
        response = requests.get(base_url, headers=headers, params=query_params)

        if response.status_code != 200:
            print(f"Failed to retrieve page {page + 1}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        job_cards = soup.find_all("div", class_="job_seen_beacon")

        for job in job_cards:
            title_elem = job.find("h2", class_="jobTitle")
            company_elem = job.find("span", class_="companyName")
            location_elem = job.find("div", class_="companyLocation")

            title = title_elem.get_text(strip=True) if title_elem else "N/A"
            company = company_elem.get_text(strip=True) if company_elem else "N/A"
            location = location_elem.get_text(strip=True) if location_elem else "N/A"

            file.write(f"Job Title: {title}\n")
            file.write(f"Company: {company}\n")
            file.write(f"Location: {location}\n")
            file.write("-" * 40 + "\n")

        print(f"Page {page + 1} scraped successfully.")
        time.sleep(2)  # Delay to prevent overwhelming the server


Failed to retrieve page 1
Failed to retrieve page 2
Failed to retrieve page 3
Failed to retrieve page 4
Failed to retrieve page 5


In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import os

def scrape_indeed_jobs(query, location, num_pages=5):
    """
    Scrape Indeed job listings for a given query and location.

    Args:
        query (str): Job title or keyword to search for
        location (str): Location to search in
        num_pages (int): Number of pages to scrape (default: 5)

    Returns:
        list: List of job listings (each as a dictionary)
    """
    # Format the search parameters for the URL
    formatted_query = query.replace(' ', '+')
    formatted_location = location.replace(' ', '+').replace(',', '%2C')

    # List to store all job listings
    all_jobs = []

    # User-Agent to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Loop through pages
    for page in range(num_pages):
        # Calculate the start parameter (10 jobs per page)
        start = page * 10

        # Create the URL for the current page
        url = f"https://www.indeed.com/jobs?q={formatted_query}&l={formatted_location}&start={start}"

        print(f"Scraping page {page+1}/{num_pages}: {url}")

        try:
            # Make the request
            response = requests.get(url, headers=headers)

            # Check if request was successful
            if response.status_code == 200:
                # Parse the HTML
                soup = BeautifulSoup(response.text, 'html.parser')

                # Find all job cards
                job_cards = soup.find_all('div', class_='job_seen_beacon')

                if not job_cards:
                    print(f"No job cards found on page {page+1}. Indeed may have changed their HTML structure or is blocking scraping.")
                    continue

                # Extract data from each job card
                for card in job_cards:
                    job_data = {}

                    # Extract job title
                    title_element = card.find('a', class_='jcs-JobTitle')
                    if title_element:
                        job_data['title'] = title_element.text.strip()

                        # Extract job URL
                        job_url = title_element.get('href')
                        if job_url and job_url.startswith('/'):
                            job_data['url'] = f"https://www.indeed.com{job_url}"
                        else:
                            job_data['url'] = job_url
                    else:
                        job_data['title'] = "N/A"
                        job_data['url'] = "N/A"

                    # Extract company name
                    company_element = card.find('span', class_='companyName')
                    job_data['company'] = company_element.text.strip() if company_element else "N/A"

                    # Extract location
                    location_element = card.find('div', class_='companyLocation')
                    job_data['location'] = location_element.text.strip() if location_element else "N/A"

                    # Extract salary if available
                    salary_element = card.find('div', class_='metadata salary-snippet-container')
                    job_data['salary'] = salary_element.text.strip() if salary_element else "Not specified"

                    # Extract job snippet/description
                    description_element = card.find('div', class_='job-snippet')
                    job_data['description'] = description_element.text.strip() if description_element else "N/A"

                    # Extract date posted
                    date_element = card.find('span', class_='date')
                    job_data['date_posted'] = date_element.text.strip() if date_element else "N/A"

                    # Add to our list of jobs
                    all_jobs.append(job_data)

                # Add a delay between requests to avoid being blocked
                time.sleep(random.uniform(2, 5))

            else:
                print(f"Failed to fetch page {page+1}. Status code: {response.status_code}")

        except Exception as e:
            print(f"Error scraping page {page+1}: {str(e)}")

    print(f"Scraped a total of {len(all_jobs)} job listings.")
    return all_jobs

def save_to_csv(jobs, filename="indeed_jobs.csv"):
    """Save job listings to CSV file"""
    if not jobs:
        print("No jobs to save.")
        return

    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            # Get the keys from the first job to use as field names
            fieldnames = jobs[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for job in jobs:
                writer.writerow(job)

        print(f"Successfully saved {len(jobs)} jobs to {filename}")
    except Exception as e:
        print(f"Error saving to CSV: {str(e)}")

def save_to_txt(jobs, filename="indeed_jobs.txt"):
    """Save job listings to TXT file"""
    if not jobs:
        print("No jobs to save.")
        return

    try:
        with open(filename, 'w', encoding='utf-8') as txtfile:
            txtfile.write(f"INDEED JOB LISTINGS\n")
            txtfile.write(f"Total Jobs Found: {len(jobs)}\n")
            txtfile.write("="*50 + "\n\n")

            for i, job in enumerate(jobs, 1):
                txtfile.write(f"Job #{i}\n")
                txtfile.write(f"Title: {job.get('title', 'N/A')}\n")
                txtfile.write(f"Company: {job.get('company', 'N/A')}\n")
                txtfile.write(f"Location: {job.get('location', 'N/A')}\n")
                txtfile.write(f"Salary: {job.get('salary', 'Not specified')}\n")
                txtfile.write(f"Date Posted: {job.get('date_posted', 'N/A')}\n")
                txtfile.write(f"Description: {job.get('description', 'N/A')}\n")
                txtfile.write(f"URL: {job.get('url', 'N/A')}\n")
                txtfile.write("-"*50 + "\n\n")

        print(f"Successfully saved {len(jobs)} jobs to {filename}")
    except Exception as e:
        print(f"Error saving to TXT: {str(e)}")

def main():
    # Get user input
    query = input("Enter job title or keywords (default: 'data scientist'): ") or "data scientist"
    location = input("Enter location (default: 'Bolton, MA'): ") or "Bolton, MA"
    num_pages = int(input("Enter number of pages to scrape (default: 5): ") or 5)

    # Scrape the jobs
    jobs = scrape_indeed_jobs(query, location, num_pages)

    if jobs:
        # Save to both CSV and TXT formats
        save_to_csv(jobs)
        save_to_txt(jobs)
    else:
        print("No jobs were found or there was an issue with scraping.")

if __name__ == "__main__":
    main()


SyntaxError: invalid syntax (<ipython-input-17-557a054a6150>, line 178)